# Preventing a Database Reconstruction Attack with Differentially Private Synthetic Data

## About the Attack
This notebook is based off of a similar [notebook](https://github.com/Trusted-AI/adversarial-robustness-toolbox/blob/main/notebooks/attack_database_reconstruction.ipynb) from IBM's adversarial robustness toolkit (ART).

It uses the [DatabaseReconstruction](https://github.com/Trusted-AI/adversarial-robustness-toolbox/blob/main/art/attacks/inference/reconstruction/white_box.py) inference attack.

## Attack Methodology
The attacker has access to a pretrained model trained on a full dataset, and all of the "public" data except a single specific missing row. This is akin to the following real world scenario: a single user requests that their data be disincluded from a dataset, and in response the data owner deletes the user's data from the public database in order to "protect" their information. The attack uses the given pretrained model as an estimator and infers the missing row.

In [1]:
from snsynth.pytorch.nn.patectgan import PATECTGAN
from snsynth.quail import QUAILSynthesizer
from diffprivlib.models import LogisticRegression as DPLR
from snsynth.pytorch.pytorch_synthesizer import PytorchDPSynthesizer

import numpy as np
import pandas as pd

/Users/lucasrosenblatt/opt/miniconda3/envs/smartnoise-tests/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


### Dataset
We use the Car Evaluation dataset [https://archive.ics.uci.edu/ml/datasets/car+evaluation] 

### Creating a Differentially Private synthesizer
We create a differentially private synthetic dataset with PATECTGAN and QUAIL here (we will use this dataset to train our model).

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from load_data import load_data
loaded_datasets = load_data(['car'])
real_data = loaded_datasets['car']['data']

# PATECTGAN generated samples
def QuailClassifier(epsilon):
            return DPLR(epsilon=epsilon, data_norm=5.02)

def QuailSynth(epsilon):
    return PytorchDPSynthesizer(epsilon=epsilon, preprocessor=None,
                    gan=PATECTGAN(epsilon=epsilon, loss='cross_entropy', batch_size=50, pack=1, sigma=5.0))

synth = QUAILSynthesizer(10.0, QuailSynth, QuailClassifier, 'class', eps_split=0.8)                
synth.fit(real_data)
sample_size = real_data.shape[0]
# Note we withhold 4 samples for support samples, detailed below
synthetic_data = synth.sample(sample_size-4)


Memory consumed by car:96896


/Users/lucasrosenblatt/opt/miniconda3/envs/smartnoise-tests/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/Users/lucasrosenblatt/opt/miniconda3/envs/smartnoise-tests/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/Users/lucasrosenblatt/opt/miniconda3/envs/smartnoise-tests/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/Users/lucasrosenblatt/opt/miniconda3/envs/smartnoise-tests/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 0.25

In [3]:
# Here we add a dummy support row for each class
# Occasionally, our synthesizer fails to generate a row of a certain class (usually "1" and "3")
# and so breaks the ART DatabaseReconstructer (which needs matching dimensions for target class)
# This is (very) bad practice generally, but ensures that our example does not break
synthetic_support_rows = pd.DataFrame(columns=synthetic_data.columns, dtype=np.int64)
for i in range(0,2):
    synthetic_support_rows = synthetic_support_rows.append({'buying':0,
                                                            'lug_boot': 0,
                                                            'doors': 0, 
                                                            'maint': 0, 
                                                            'persons': 0,
                                                            'safety': 0, 
                                                            'class': 1}, ignore_index=True)
    synthetic_support_rows = synthetic_support_rows.append({'buying':0,
                                                            'lug_boot': 0,
                                                            'doors': 0, 
                                                            'maint': 0, 
                                                            'persons': 0,
                                                            'safety': 0, 
                                                            'class': 3}, ignore_index=True)
# Add support rows
synthetic_data = synthetic_data.append(synthetic_support_rows)
# Reset index
synthetic_data.reset_index(drop=True, inplace=True)

### Train/test splits for both real and synthetic data

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
seed = 42

def return_train_splits(df, target, test_size=0.2):
    Xy_train, Xy_test = train_test_split(df, test_size=test_size, stratify = df[target], random_state = seed)

    X_train = Xy_train.drop(target, axis=1)
    y_train = Xy_train[target]

    X_test = Xy_test.drop(target, axis=1)
    y_test = Xy_test[target]
    return X_train, y_train, X_test, y_test 

X_train, y_train, X_test, y_test = return_train_splits(real_data, 'class')
X_train_synth, y_train_synth, X_test_synth, y_test_synth = return_train_splits(synthetic_data, 'class')

### Our classifiers (non-private and private)
As of right now, it seems that the IBM ART DatabaseReconstruction attack only works on the GaussianNB sklearn implementation, so we use this model for comparison. We train a non-private and private version of this model (the private version of this model is trained on the synthetic dataset we generated earlier)

### Results (classifier ML utility)
GaussianNB is not the ideal modeling approach for the car dataset, so neither the non-private nor the private models perform particularly well. They do, however, perform comparably (overall accuracy is ~0.64 for non-private versurs ~0.66 for private). We sometimes see this counter-intuitive result when the private synthetic data trained model is able to generalize slightly better than the non-private model.

Note that the private model is evaluated as *TSTR* (train on synthetic data, test on real data).

In [10]:
from sklearn.naive_bayes import GaussianNB
from art.estimators.classification.scikitlearn import ScikitlearnGaussianNB

clf = GaussianNB()
clf.fit(X_train, y_train)    

art_classifier = ScikitlearnGaussianNB(clf)

clf_synth = GaussianNB()
clf_synth.fit(X_train_synth, y_train_synth)    

art_classifier_synth = ScikitlearnGaussianNB(clf_synth)

def evaluate (clf, X_test, y_test):
    y_pred = clf.predict(X_test)
    print(classification_report(y_test, y_pred))
    y_probs = clf.predict_proba(X_test)
    print(accuracy_score(y_test, y_pred))

evaluate(clf_synth, X_test, y_test)
evaluate(clf, X_test, y_test)

              precision    recall  f1-score   support

           0       0.45      0.23      0.31        77
           1       0.00      0.00      0.00        14
           2       0.74      0.94      0.83       242
           3       0.00      0.00      0.00        13

    accuracy                           0.71       346
   macro avg       0.30      0.29      0.28       346
weighted avg       0.62      0.71      0.65       346

0.708092485549133
              precision    recall  f1-score   support

           0       0.56      0.06      0.12        77
           1       0.00      0.00      0.00        14
           2       0.84      0.84      0.84       242
           3       0.14      1.00      0.25        13

    accuracy                           0.64       346
   macro avg       0.38      0.48      0.30       346
weighted avg       0.71      0.64      0.62       346

0.6416184971098265


/Users/lucasrosenblatt/opt/miniconda3/envs/smartnoise-tests/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lucasrosenblatt/opt/miniconda3/envs/smartnoise-tests/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Random target row selection
From IBM: "We now select a row from the training dataset that we will remove. This is the target row which the attack will seek to reconstruct. The attacker will have access to x_public and y_public."

In [7]:
target_row = np.random.choice(X_train.index, 1, replace=False)

x_public = X_train.drop(target_row)
y_public = y_train.drop(target_row)

x_public_synth = X_train_synth.drop(target_row)
y_public_synth = y_train_synth.drop(target_row)

### The Database Reconstruction attack
Here we create 2 attacks: one with the non-private classifier, and the other with the private (synthetic data trained) classifier. 

Note that the private classifier is evaluated on attempting two reconstructions - the first, as though the synthetic data was released (this is an odd, unlikely scenario - hard to imagine a user requesting that fake data be disincluded from fake data - but worth looking at as it demonstrates how effective the privatization has been). The second scenario is against the real data (as though many users gave their permission to release data, but we want to protect users who opted out).

In [8]:
from art.attacks.inference.reconstruction import DatabaseReconstruction

# Non-private Classifier
dbrecon = DatabaseReconstruction(art_classifier)

x_pub_numpy = x_public.to_numpy()
y_pub_numpy = y_public.to_numpy()
x_pub_numpy_synth = x_public_synth.to_numpy()
y_pub_numpy_synth = y_public_synth.to_numpy()

# Reconstructing the missing row
x, y = dbrecon.reconstruct(x_pub_numpy, y_pub_numpy)

# Private classifier reconstruction attack
dbrecon_synth = DatabaseReconstruction(art_classifier_synth)

# Reconstructing the "missing row" from the synthetic dataset
x_synth_synth, y_synth_synth = dbrecon_synth.reconstruct(x_pub_numpy_synth, y_pub_numpy_synth)

# Reconstructing the missing row from the real dataset
x_synth_real, y_synth_real = dbrecon_synth.reconstruct(x_pub_numpy, y_pub_numpy)

### Results
The reconstruction attack is **highly** successful when the non-private classifier is released along with the real data - it is essentially able to reconstruct the missing row perfectly. With differential privacy, the attack is completely thwarted. Not only does the example row pass the sniff test, but the RMSE is orders of magnitude higher than without privatization (a good thing in this case!).

In [9]:
np.set_printoptions(suppress=True)
X_train_np = X_train_synth.to_numpy()
original = X_train.loc[target_row].to_numpy()
print('Original row:' + str(original))
print('Reconstructed row (no DP):' + str(x))
print('Reconstructed row (WITH DP) (synthetic data):' + str(x_synth_synth))
print('Reconstructed row (WITH DP) (real data):' + str(x_synth_real))
print()
print("Inference RMSE (without differential privacy): {}".format(
    np.sqrt(((original - x) ** 2).sum() / X_train.shape[1])))

print("Inference RMSE (WITH differential privacy) (synthetic data): {}".format(
    np.sqrt(((original - x_synth_synth) ** 2).sum() / X_train.shape[1])))

print("Inference RMSE (WITH differential privacy) (real data): {}".format(
    np.sqrt(((original - x_synth_real) ** 2).sum() / X_train.shape[1])))

Original row:[[3 1 0 1 1 0]]
Reconstructed row (no DP):[[ 3.00000009  0.99999955 -0.00000016  0.99999999  1.         -0.00000001]]
Reconstructed row (WITH DP) (synthetic data):[[1.99999304 1.99999983 0.00000016 1.0000024  0.00000018 1.00000959]]
Reconstructed row (WITH DP) (real data):[[-0.83311108 -0.83311108  0.89986564 -0.73276206  0.1946248   0.52915717]]

Inference RMSE (without differential privacy): 1.9742889242304893e-07
Inference RMSE (WITH differential privacy) (synthetic data): 0.8164998879540726
Inference RMSE (WITH differential privacy) (real data): 1.9490979010415337
